In [78]:
import os
import cv2
import pyautogui
from joblib import load
from collections import deque
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

In [79]:
def detect_and_crop_eyes(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect eyes in the frame
        eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)
        print(f"Detected {len(eyes)} eyes in frame {frame_count}")

        for (ex, ey, ew, eh) in eyes:
            # Crop the eye region from the frame
            eye_img = frame[ey:ey + eh, ex:ex + ew]

            # Save the cropped eye image
            eye_img_path = os.path.join(output_dir, f'eye_{frame_count}.jpg')
            cv2.imwrite(eye_img_path, eye_img)
            print(f"Saved eye image: {eye_img_path}")
            frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

video_path = 'WhatsApp Video 2024-07-29 at 09.54.56_cb79e8e2.mp4'
output_dir = 'output_eye_images'
detect_and_crop_eyes(video_path, output_dir)


Detected 3 eyes in frame 0
Saved eye image: output_eye_images\eye_0.jpg
Saved eye image: output_eye_images\eye_1.jpg
Saved eye image: output_eye_images\eye_2.jpg
Detected 3 eyes in frame 3
Saved eye image: output_eye_images\eye_3.jpg
Saved eye image: output_eye_images\eye_4.jpg
Saved eye image: output_eye_images\eye_5.jpg
Detected 3 eyes in frame 6
Saved eye image: output_eye_images\eye_6.jpg
Saved eye image: output_eye_images\eye_7.jpg
Saved eye image: output_eye_images\eye_8.jpg
Detected 3 eyes in frame 9
Saved eye image: output_eye_images\eye_9.jpg
Saved eye image: output_eye_images\eye_10.jpg
Saved eye image: output_eye_images\eye_11.jpg
Detected 4 eyes in frame 12
Saved eye image: output_eye_images\eye_12.jpg
Saved eye image: output_eye_images\eye_13.jpg
Saved eye image: output_eye_images\eye_14.jpg
Saved eye image: output_eye_images\eye_15.jpg
Detected 3 eyes in frame 16
Saved eye image: output_eye_images\eye_16.jpg
Saved eye image: output_eye_images\eye_17.jpg
Saved eye image: o

In [80]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (50, 50))  # Resize all images to 50x50
            images.append(img)
            labels.append(label)
    return images, labels

def prepare_dataset(left_dir, right_dir, up_dir, down_dir):
    left_images, left_labels = load_images_from_folder(left_dir, 0)
    right_images, right_labels = load_images_from_folder(right_dir, 1)
    up_images, up_labels = load_images_from_folder(up_dir, 2)
    down_images, down_labels = load_images_from_folder(down_dir, 3)

    images = left_images + right_images + up_images + down_images
    labels = left_labels + right_labels + up_labels + down_labels

    images = np.array(images)
    labels = np.array(labels)

    images = images.reshape((images.shape[0], -1))

    return images, labels

In [81]:
# Directories containing cropped eye images
left_dir = r'myeyedataset/left'
right_dir = r'myeyedataset\right'
up_dir = r'myeyedataset\up'
down_dir = r'myeyedataset\down'

X, y = prepare_dataset(left_dir, right_dir, up_dir, down_dir)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_clf = SVC(kernel='linear')
svm_clf.fit(X_train, y_train)

y_pred = svm_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Save the trained model
model_path = 'eye_direction_svm_model.joblib'
joblib.dump(svm_clf, model_path)

Accuracy: 0.97


['eye_direction_svm_model.joblib']

In [82]:
def extract_eye_region(frame):
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.3, 5)
    
    if len(eyes) > 0:
        (ex, ey, ew, eh) = eyes[0]
        eye_region = frame[ey:ey+eh, ex:ex+ew]
        return eye_region
    else:
        return None

def extract_features(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (50, 50))  # Resize to 70x70 pixels
    flattened = resized.flatten()
    return flattened

In [83]:
def predict_direction(frame, model):
    features = extract_features(frame)
    
    if features is not None:
        prediction = model.predict([features])[0]
        direction_labels = {0: 'left', 1: 'right', 2: 'up', 3: 'down'}
        
        if prediction in direction_labels:
            predicted_direction = direction_labels[prediction]
            return predicted_direction
        else:
            print(f"Unexpected prediction value: {prediction}")
            return None
    else:
        return None

In [84]:
def move_cursor_with_eye_direction(video_path, model):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    screen_width, screen_height = pyautogui.size()
    cursor_step = 50  # Number of pixels to move the cursor
    
    direction_buffer = deque(maxlen=5)  # Buffer to hold the last 5 directions for smoothing
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        eye_region = extract_eye_region(frame)
        
        if eye_region is not None:
            direction = predict_direction(eye_region, model)
            
            if direction is not None:
                direction_buffer.append(direction)
                
                # Use majority voting to smooth the direction
                if len(direction_buffer) == direction_buffer.maxlen:
                    smoothed_direction = max(set(direction_buffer), key=direction_buffer.count)
                    
                    # Only move cursor if there's a clear majority
                    if direction_buffer.count(smoothed_direction) > len(direction_buffer) // 2:
                        if smoothed_direction == 'left':
                            pyautogui.moveRel(-cursor_step, 0)
                        elif smoothed_direction == 'right':
                            pyautogui.moveRel(cursor_step, 0)
                        elif smoothed_direction == 'up':
                            pyautogui.moveRel(0, -cursor_step)
                        elif smoothed_direction == 'down':
                            pyautogui.moveRel(0, cursor_step)
                    
                    cv2.putText(frame, f"Direction: {smoothed_direction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                else:
                    cv2.putText(frame, f"Direction: {direction}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        cv2.imshow('Frame', frame)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

video_path = r'WhatsApp Video 2024-07-29 at 09.54.56_cb79e8e2.mp4'
model_path = r'eye_direction_svm_model.joblib'
svm_clf = load(model_path)

move_cursor_with_eye_direction(video_path, svm_clf)
